In [1]:
class Leaf:
    def __init__(self,origin,left,right=''):
        self.origin = origin
        if len(left) == 0:
            self.left = '$'
            self.right = 1
        else:
            self.left = left
            self.right = Leaf(None,right)
    
    def __repr__(self): return '(' + repr(self.left) + ',' + repr(self.right) + ')'

    def __eq__(self,other): return self[0] == other[0]
    # def __eq__(self,other): 
    #     if type(other) == str: return self[:min(len(other),len(self.left))] ==  other[:min(len(other),len(self.left))]
    #     return self[:min(len(other.left),len(self.left))] == other[:min(len(other.left),len(self.left))]

    def __hash__(self): return hash(self[0])

    def __getitem__(self,index): return self.left[index]

    def __len__(self):
        if self.left == '$': return 0
        return len(self.left)+len(self.right)

In [17]:
class Branch:
    def __init__(self):
        self.b = {}
        self.reads = []

    def __repr__(self): 
        s = ''
        for a in self.b.values(): 
            s += repr(a) 
        return s

    def __str__(self):
        s = '' 
        for i in range(len(list(self.b.values()))-1):s+=str(list(self.b.values())[i])+'\n'
        return s+str(list(self.b.values())[-1])
    
    def __getitem__(self,index): return self.b[index]
    
    def __contains__(self,other): return other in self.b

    def __len__(self): return len(self.b)

    def is_shallow(self):
        for x in self.b.values():
            if type(x.right) == Leaf or x.left == '$': continue
            else: 
                for y in x.right.b.values():
                    if type(y.right) == Leaf or y.left == '$': continue
                    else: return False
        return True

    def add(self,leaf,read):
        if leaf in self.b:
            if self.b[leaf].left == '$':
                self.b[leaf].right+=1
                return
            i = 0
            left = ''
            while i < min(len(leaf.left),len(self.b[leaf].left)) and leaf.left[i] == self.b[leaf][i]: 
                left += leaf.left[i]
                i += 1
            if type(self.b[leaf].right) == Branch:
                nleaf = Leaf(leaf.origin,leaf.left[i:])
                if len(self.b[leaf].left[i:]):
                    branch = Branch()
                    l = Leaf(self.b[leaf].origin,self.b[leaf].left[i:])
                    l.right = self.b[leaf].right
                    branch.b[l] = l
                    branch.reads += [l.origin]
                    self.b[leaf].right = branch
                self.b[leaf].right.add(nleaf,leaf.origin)
            else: 
                branch = Branch()
                branch.add(Leaf(self.b[leaf].origin,self.b[leaf].left[i:]),self.b[leaf].origin)
                branch.add(Leaf(leaf.origin,leaf.left[i:]),leaf.origin)
                self.b[leaf].right = branch
            self.b[leaf].left = left
            self.reads += [read]
        else: 
            self.b[leaf] = leaf
            self.reads += [read]

In [3]:
class Sequence:
    def __init__(self,seq=None,initus=None,terminus=None):
        if seq is None: self.seq = ''
        else: self.seq = seq
        if initus is None: self.initus = ''
        else: self.initus = initus
        if terminus is None: self.terminus = ''
        else: self.terminus = terminus

In [13]:
class Sequitur:
    def __init__(self,reads):
        self.branch = Branch()
        self.reads = reads
        for read in reads: 
            for i in range(len(read)): self.branch.add(Leaf(read,read[i:]),read)
        self.sequence = ''

    def _traverse(self,context):
        branch = self.branch.b[context[0]]
        context = context[len(branch.left):]
        while len(context) > 0:
            # if q[0] not in branch.right:
            #     print sequence.seq
            #     print(branch)
            branch = branch.right[context[0]]
            context = context[len(branch.left):]
        return branch

    def sequitur(self,sequence,seqs,stops=None,initial=None,terminal=None):
        i = 1
        context = sequence.seq[-i]
        options = []
        while i <= len(sequence.seq):
            branch = self._traverse(context)
            i += 1
            context = sequence.seq[-i:]
            # if stops:
            #     if stops[0].seq.startswith(q):
            #         i_ = 0
            #         br = branch
            #         while type(br.right) == Branch and len(q) + i_+ len(br.left) <= len(stops[0].seq):
            #             q_ = stops[0].seq[len(q)+i_]
            #             if q_ in br.right: br = br.right[q_]
            #             else: br = br.right['$']
            #             i_ += len(br.left)
            #         if type(br.right) == Branch: br = br.right['$']
            #         if stops[0].seq.startswith(br.origin): terminal = (br,q)
            #         elif br.origin.endswith(stops[0].seq[stops[0].seq.find(stops[0].seq[:len(q)+i_]):]):
            #             stops = list(stops)
            #             stops[0].seq = br.origin[:-len(stops[0].seq[stops[0].seq.find(stops[0].seq[:len(q)+i_]):])] + stops[0].seq
            #             stops[0].terminus = br.origin
            #             stops[1].add(br.origin)
            #             stops = tuple(stops)
            #     if stops[0].seq in sequence.seq:
            #         seqs.update(stops[1])
            #         options += stops[2]
            #         stops = None
            # if type(branch.right) == Leaf or branch.right.is_shallow(): break
            # if i > len(sequence.seq) and '$' in branch.right.b and branch.right['$'].origin == sequence.seq:
            #     for h in branch.right.b:
            #         if sequence.seq in h.origin and h.origin not in seqs:
            #             sequence.seq = h.origin
            #             sequence.initus = h.origin
            #             sequence.terminus = h.origin
            #             seqs.add(h.origin)
            #             i = 1
            #             q = sequence.seq[-i:]
            #             break
            # branch = self.branch.b[q[0]]
        if not initial:
            if type(branch.right) == Leaf: options += [branch]
            else:
                for a in branch.right.b:
                    if a.left != '$': options += [a]
                    elif sequence.seq in a.origin:
                        sequence.seq = a.origin
                        sequence.initus = a.origin
                        sequence.terminus = a.origin
                        seqs.add(a.origin)
                    elif a.origin in sequence.seq:
                        seqs.add(a.origin)
        else: options = initial
        options_ = []
        for o in options:
            if type(o.right) == Branch:
                for r in o.right.b.values(): 
                    if r.left == '$':
                        if len(o.right.b.values()) == 1: 
                            l = Leaf(r.origin,o.left)
                            options += [l]
                        continue
                    l = Leaf(r.origin,o.left+r.left)
                    options += [l]
                continue
            if stops and o.origin in stops[1]:
                if sequence.seq.endswith(stops[0].seq[:stops[0].seq.find(q)+len(q)]): 
                    sequence.seq += stops[0].seq[stops[0].seq.find(q)+len(q):]
                    sequence.terminus = stops[0].terminus
                    seqs.update(stops[1])
                    options += stops[2]
                    return sequence, seqs, options, False, terminal
            if sequence.seq in o.origin:
                sequence.seq = o.origin
                sequence.initus = o.origin
                sequence.terminus = o.origin
                seqs.add(o.origin)
                continue
            if sequence.seq.endswith(o.origin[:o.origin.find(q)+len(q)]) and o.origin not in seqs: options_ += [o]
        options = options_
        if stops is not None and len(options) == 0:
            if terminal is not None and sequence.seq.endswith(terminal[1]) and stops[0].seq.startswith(terminal[1]): 
                i = 1
                # q = terminal[0].origin
                q = sequence.seq[-(len(terminal[1])+1)] + terminal[0].origin
                branch = self.branch.b[q[0]]
                q = q[len(branch.left):]
                while len(q) > 0:
                    if q[0] not in branch.right: break
                    branch = branch.right[q[0]]
                    q = q[len(branch.left):]
                if sequence.seq.endswith(branch.origin.split(terminal[1])[0]+terminal[1]) and terminal[0].origin.startswith(terminal[1]+branch.origin.split(terminal[1])[1]):
                # if len(q) == 0 or type(branch) == Leaf:
                    sequence.seq += stops[0].seq[len(terminal[1]):]
                    sequence.terminus = stops[0].terminus
                    seqs.update(stops[1])
                    options += stops[2]
                    return sequence, seqs, options, False, None
            if len(seqs.intersection(set([s.origin for s in stops[2]]))):
                # stops[0] is the initus
                rem = []
                for s in stops[2]:
                    if s.origin in seqs.intersection(set([s.origin for s in stops[2]])): rem += [s]
                stops = list(stops)
                for r in rem: stops[2].remove(r)
                if len(stops[2]) == 1:
                    o = stops[2].pop()
                    stops[1].add(o.origin)
                    stops[0].seq += o.left
                    stops[0].terminus = o.origin
                stops = tuple(stops)
                return sequence, seqs, options, False, terminal
                # i_ = 1
                # while stops[0].terminus[-i_:] in sequence.initus: i_+=1
                # if sequence.initus.startswith(stops[0].terminus[-i_:]):
                #     sequence.initus = stops[0].initus
                #     sequence.seq = stops[0].seq[:-i_] + sequence.seq
                #     seqs.update(stops[1])
        extension = False
        if len(options) == 1: 
            o = options.pop()
            seqs.add(o.origin)
            sequence.seq += o.origin[o.origin.find(q)+len(q):]
            sequence.terminus = o.origin
            extension = True
        return sequence, seqs, options, extension, terminal

    def assemble(self,start=None,seed=None,correct_sequence=None):
        import random
        if seed is not None: random.seed(seed)

        if start is None or start not in self.reads: start = random.choice(self.reads)
        print('Initus:',start)
        self.sequence = self.sequitur(Sequence(start,start,start),{start})
        while len(set(self.reads).difference(self.sequence[1])) > 0: 
            self.sequence = self.sequitur(self.sequence[0],self.sequence[1],terminal=self.sequence[4])
            if len(self.sequence[2]):
                nseq = random.choice(self.sequence[2]).origin
                nseq = self.sequitur(Sequence(nseq,nseq,nseq),{nseq},stops=self.sequence,terminal=self.sequence[4])
                if self.sequence[0].seq == 'AGTCTTGATGCTCGGGGGACACTTCTTTAAGCTCGGTGTGGTGGGCACGACCCTGGACGCGCGACGAAGCTAAGTTTGCAGTAATTAACCGACATCTTTGTGAACCGACCCACATTTGACGGTACGCTACCGCAACGGTATGTGTTAATGGAACAGACTTGCTTATGTGGACGTTGTATAGGGATATTACGTTACGCGTTAACCGATACATACTGGTTTCTCTCCAGTGGAGGTCTTGGTTGCCTCTAGTTTCTACGATATACTCATGGTAGTGTAACGCATAATCGAAGAGGGTCCTCCCATCTCCTGTGATGCATGGTGTGCTTACTGGGATGAATGCGCCGCAAGTAGCAGGTCCCGGCGTGGATACCTGATAGATGGTGACTAGCATGTACAAGTAACCTTGTCTATTGAGCTTCGAGGATGCATACAAGCCCACCCGCAGCCGC':
                    print()
                while nseq[3] and len(nseq[2]) == 0: nseq = self.sequitur(nseq[0],nseq[1],stops=self.sequence)
                self.sequence = self.sequitur(nseq[0],nseq[1],initial=nseq[2],terminal=self.sequence[4])
            if not self.sequence[3] and len(self.sequence[2]) == 0 and len(set(self.reads).difference(self.sequence[1])):
                nseq = random.choice((list(set(self.reads).difference(self.sequence[1]))))
                while nseq in self.sequence[0].seq and len(set(self.reads).difference(self.sequence[1])): 
                    self.sequence[1].add(nseq)
                    if not len(set(self.reads).difference(self.sequence[1])): continue
                    nseq = random.choice((list(set(self.reads).difference(self.sequence[1]))))
                if nseq not in self.sequence[0].seq:
                    nseq = self.sequitur(Sequence(nseq,nseq,nseq),{nseq},stops=self.sequence,terminal=self.sequence[4])
                    while nseq[3] and len(nseq[2]) == 0: nseq = self.sequitur(nseq[0],nseq[1],stops=self.sequence,terminal=self.sequence[4])
                    self.sequence = self.sequitur(nseq[0],nseq[1],initial=nseq[2],terminal=self.sequence[4])

In [18]:
import random

n = 2000
seeds = [random.randint(0,13350) for a in range(n)]
seeds = list(set(seeds))
n = len(seeds)

correct_sequence = 'betty_bought_butter_the_butter_was_bitter_betty_bought_better_butter_to_make_the_bitter_butter_better'
print('Sequence:',correct_sequence)

successes = 0
reads = ['betty_bought_butter_th','tter_the_butter_was_','he_butter_was_bitter_','as_bitter_betty_bought','tty_bought_better_butter_t','r_butter_to_make_the_','ke_the_bitter_butter_better']
for i in range(n):
    random.seed(seeds[i])
    random.shuffle(reads)

    seq = Sequitur(reads)
    seq.assemble(seed=seeds[i])

    print('Seed:',seeds[i])
    if seq.sequence[0].seq == correct_sequence: 
        print('[SUCCESS]')
        successes+=1
    else: print('[FAILURE]')
    print('------------------------')
print('Accuracy:',successes/n*100,'%')
print('========================')

Sequence: betty_bought_butter_the_butter_was_bitter_betty_bought_better_butter_to_make_the_bitter_butter_better
Initus: as_bitter_betty_bought


KeyboardInterrupt: 

In [ ]:
import random

n = 2000
seeds = [random.randint(0,13350) for a in range(n)]
seeds = list(set(seeds))
n = len(seeds)

correct_sequence = 'she_sells_sea_shells_on_the_sea_shore'
print('Sequence:',correct_sequence)

successes = 0
reads = ['she_sells_s',
               'lls_sea_shel',
                    'ea_shells_o',
                       'shells_on_the_s',
                                  'he_sea_s',
                                      'ea_shore']
for i in range(n):
    random.seed(seeds[i])
    random.shuffle(reads)

    seq = Sequitur(reads)
    seq.assemble(seed=seeds[i])

    print('Seed:',seeds[i])
    if seq.sequence[0].seq == correct_sequence: 
        print('[SUCCESS]')
        successes+=1
    else: print('[FAILURE]')
    print('------------------------')
print('Accuracy:',successes/n*100,'%')
print('========================')

In [ ]:
import random

n = 2000
seeds = [random.randint(0,13350) for a in range(n)]
seeds = list(set(seeds))
n = len(seeds)

correct_sequence = 'you say hello world, i bellow go to hell'
print('Sequence:',correct_sequence)

successes = 0

reads = ['you say hel',' say hello wo','lo world, i be','ld, i bellow go t','ow go to hell']
for i in range(n):
    random.seed(seeds[i])
    random.shuffle(reads)

    seq = Sequitur(reads)
    seq.assemble(seed=seeds[i])

    print('Seed:',seeds[i])
    if seq.sequence[0].seq == correct_sequence: 
        print('[SUCCESS]')
        successes+=1
    else: print('[FAILURE]')
    print('------------------------')
print('Accuracy:',successes/n*100,'%')
print('========================')

In [150]:
def generate_reads(seq, k, min_overlap, max_overlap,seed=None):
    random.seed(seed)
    reads = []
    for i in range(0, len(seq)-k+1):
        # Generate a random overlap within the specified range
        overlap = random.randint(min_overlap, max_overlap)
        start = i
        end = i + k + overlap
        reads.append(seq[start:end])
    return reads

def generate_genome_sequence(n,seed=None):
    random.seed(seed)
    nucleotides = {1:'A',2:'C',3:'G',4:'T'}
    seq = ''
    for i in range(n):
        seq += nucleotides[random.randint(1,4)]
    return seq

In [ ]:
import random

n = 2000
seeds = [random.randint(0,13350) for a in range(n)]
seeds = list(set(seeds))
n = len(seeds)

seed = 42
correct_sequence = generate_genome_sequence(200,seed=seed)
print('Sequence:',correct_sequence)

successes = 0

reads = generate_reads(correct_sequence,3,4,10,seed=seed)
for i in range(n):
    print('Seed:',seeds[i])
    
    random.seed(seeds[i])
    random.shuffle(reads)

    seq = Sequitur(reads)
    seq.assemble(seed=seeds[i])

    if seq.sequence[0].seq == correct_sequence: 
        print('[SUCCESS]')
        successes+=1
    else: print('[FAILURE]')
    print('------------------------')
print('Accuracy:',successes/n*100,'%')
print('========================')

In [180]:
correct_sequence = generate_genome_sequence(2000,seed=seed)
reads = generate_reads(correct_sequence,3,4,10,seed=seed)
seq = Sequitur(reads)
seq.assemble(seed=seeds[i],correct_sequence=correct_sequence)
if seq.sequence[0].seq == correct_sequence: print('[SUCCESS]')
else: print('[FAILURE]')

Initus: AACACTCATTCGA











